In [1]:
library(Rcrawler)
library(foreach)
library(doParallel)
library(scales)
library(qdapRegex)

Loading required package: iterators

Loading required package: parallel



In [2]:
Sys.setlocale("LC_CTYPE", locale="Chinese")

[1] "Chinese (Simplified)_China.936"

In [3]:
# parallel processing
cores=detectCores()
cl <- makeCluster(cores[1]-1) #not to overload your computer
registerDoParallel(cl)

In [4]:
# CSS patterns identified from website
pattern <- c(".content_table_R a", 
             ".content_table_R td:nth-child(4)",
             ".content_table_R td:nth-child(5)",
             ".content_table_R td:nth-child(6)",
             ".content_table_R td:nth-child(7)",
             ".content_table_R td:nth-child(8)")

# Names for each field collected
pattern_names <- c("id", 
                   "wins",
                   "battles",
                   "fleet",
                   "squad",
                   "activity")

scraper <- function(page){
  target_url <- paste("https://www.navyfield.com/Community/Ranking/List.aspx?page=",
                      toString(page),"&server=1&category=4&searchType=1&searchValue=", sep = "")
  web_content <- Rcrawler::ContentScraper(Url=target_url,
                                CssPatterns = pattern,
                                PatternsName = pattern_names, 
                                ManyPerPattern = TRUE, 
                                astext = TRUE, 
                                asDataFrame = FALSE)
  output <- as.data.frame(web_content)
  return(output)
}

rank_data <- foreach(i=1:500, .combine=rbind) %dopar% {
  tempMatrix = scraper(i) 
  tempMatrix #Equivalent to finalMatrix = rbind(finalMatrix, tempMatrix)
}

In [5]:
hof_pattern <- c(".rnum", 
             ".content_table_H td:nth-child(4)",
             ".content_table_H td:nth-child(5)",
             ".content_table_H td:nth-child(6)",
             ".content_table_H td:nth-child(7)")

hof_pattern_names <- c("id", 
                   "wins_total",
                   "fleet",
                   "squad",
                   "activity")

hof_scraper <- function(page){
    target_url <- paste("https://www.navyfield.com/Community/HallofFame/List.aspx?page=",
                        toString(page),"&server=&sort=1&searchType=1&searchValue=", sep = "")
    web_content <- Rcrawler::ContentScraper(Url=target_url,
                                  CssPatterns = hof_pattern,
                                  PatternsName = hof_pattern_names, 
                                  ManyPerPattern = TRUE, 
                                  astext = TRUE, 
                                  asDataFrame = FALSE)
    output <- as.data.frame(web_content)
    return(output)
}


hof_data <- foreach(i=1:7, .combine=rbind) %dopar% {
  tempMatrix = hof_scraper(i) 
  tempMatrix 
}

hof_data$activity = rm_between(hof_data$activity, '(', ')', extract=TRUE)
hof_data <- as.data.frame(lapply(hof_data, function(x) {
  gsub("\u00A0", "", x) 
})) 
hof_data$wins <- sub("/.*", "", hof_data$wins_total)
hof_data$battles <- sub(".*/", "", hof_data$wins_total)
hof_clean <- subset(hof_data, select = c(id, fleet:battles))

In [6]:
nf_data <- rbind(rank_data, hof_clean)
nf_data$id <- trimws(nf_data$id, which = c("both"))
nf_data$fleet <- trimws(nf_data$fleet, which = c("both"))
nf_data$squad <- trimws(nf_data$squad, which = c("both"))
nf_data$activity <- trimws(nf_data$activity, which = c("both"))
nf_data$activity <- gsub(" ", "", as.character(nf_data$activity))
nf_data$activity <- gsub("PST", "", as.character(nf_data$activity))
nf_data$wins <- as.numeric(gsub(",", "", as.character(nf_data$wins)))
nf_data$battles <- as.numeric(gsub(",", "", as.character(nf_data$battles)))
nf_data$win_rate <- nf_data$wins/nf_data$battles

In [7]:
nf <- file("C:/Users/Felix/Box Sync/UW/R/nf.csv", encoding = "UTF-8")
write.csv(nf_data, file = nf, row.names = FALSE)

In [60]:
# See results
substrRight <- function(x, n){
  as.numeric(substr(x, nchar(x)-n+1, nchar(x)))
}

substrLeft <- function(x, n){
  as.numeric(substr(x, 1, n))
}

today <- as.character(Sys.Date())
current_year <- as.numeric(substr(today,1,4))
current_month <- as.numeric(substr(today,6,7))

last_three_months <- function(m,y){
    if (m < 4) {
        a = m + 9
        b = y - 1
    }
    else if (m > 3) {
        a = m - 3
        b = y
    }
    return(c(a,b))
}

fleet_power <- function(fleet_name){
    fleet_data <- subset(nf_data, fleet==fleet_name)
    attach(fleet_data)
    fleet_best <- fleet_data[order(-win_rate),]
    fleet_best$win_rate <- percent(fleet_best$win_rate, accuracy = 0.1)
    detach(fleet_data)
    if (current_month > 3){
        return(head(subset(fleet_best,battles > 5000 & substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>last_three_months(current_month,current_year)[1]),20))
    } else if (current_month < 4){
        return(head(subset(fleet_best,battles > 5000 & 
                           (substrRight(fleet_best$activity,4)>last_three_months(current_month,current_year)[2] | 
                           (substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>=last_three_months(current_month,current_year)[1])
                           )
                          ),20))
    }  
}

squad_power <- function(fleet_name){
    fleet_data <- subset(nf_data, squad==fleet_name)
    attach(fleet_data)
    fleet_best <- fleet_data[order(-win_rate),]
    fleet_best$win_rate <- percent(fleet_best$win_rate, accuracy = 0.1)
    detach(fleet_data)
    if (current_month > 3){
        return(head(subset(fleet_best,battles > 5000 & substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>last_three_months(current_month,current_year)[1]),20))
    } else if (current_month < 4){
        return(head(subset(fleet_best,battles > 5000 & 
                           (substrRight(fleet_best$activity,4)>last_three_months(current_month,current_year)[2] | 
                           (substrRight(fleet_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(fleet_best$activity,2)>=last_three_months(current_month,current_year)[1])
                           )
                          ),20))
    }  
}

In [57]:
# NF players that win and/or disconnect most of the time (<_<)
# Only players active in last 3 months and have had 5000+ battles on record are listed
attach(nf_data)
nf_best <- nf_data[order(-win_rate),]
nf_best$win_rate <- percent(nf_best$win_rate, accuracy = 0.1)
detach(nf_data)
if (current_month > 3){
    head(subset(nf_best,battles > 5000 & substrRight(nf_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(nf_best$activity,2)>=last_three_months(current_month,current_year)[1]),20)
} else if (current_month < 4){
    head(subset(nf_best,battles > 5000 & 
                (substrRight(nf_best$activity,4)==last_three_months(current_month,current_year)[2] & substrLeft(nf_best$activity,2)>=last_three_months(current_month,current_year)[1]) |
                substrRight(nf_best$activity,4)>last_three_months(current_month,current_year)[2]
               ),20)
}

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1106,joooooo,6282,7864,REGALIA,-,07.12.2021,79.9%
934,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
1736,baozi77,4310,5585,-,-,07.12.2021,77.2%
1478,pr_ingress,5001,6487,TEAM PRO BOTS,-,07.12.2021,77.1%
243,jedizorro,13161,17264,Dango Daikazoku,-,07.12.2021,76.2%
10009,yyss2221,20543,27431,海风舰队,海風奔馳,07.12.2021,74.9%
890,betep,7262,9902,Russian Alliance,Nakhimov,07.13.2021,73.3%
247,k435345,13128,18023,-,-,07.04.2021,72.8%
244,greattak,13158,18252,-,-,07.12.2021,72.1%


In [61]:
# 深海舰队
shenhai <- fleet_power('深海舰队')
shenhai

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1271,b911753382a,5670,9696,深海舰队,-,07.12.2021,58.5%
1500,zcx19740112,4937,8454,深海舰队,深渊,05.06.2021,58.4%
1887,chalex001,4020,7328,深海舰队,-,07.12.2021,54.9%
1250,erissoft,5749,10676,深海舰队,深渊,07.08.2021,53.8%
1314,wjsh2020,5551,10627,深海舰队,-,07.12.2021,52.2%
1518,guanguanktv,4884,9366,深海舰队,深蓝战队,07.12.2021,52.1%
189,connan_add,14488,27823,深海舰队,-,07.12.2021,52.1%
2166,zbyzw2001,3346,6449,深海舰队,深渊,07.13.2021,51.9%
2111,ddra721027,3457,6719,深海舰队,-,07.12.2021,51.5%


In [62]:
# 兔子舰队
rabbit <- fleet_power('兔子.CHN.')
rabbit

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1117,la2gajaltmot,6251,9082,兔子.CHN.,World,07.12.2021,68.8%
146,tskspree,15646,22975,兔子.CHN.,World,07.12.2021,68.1%
3,laoji,37493,55264,兔子.CHN.,-,07.12.2021,67.8%
1610,hina272k,4631,6968,兔子.CHN.,Bravery,07.12.2021,66.5%
237,dreamcaca0,13328,20551,兔子.CHN.,World,07.13.2021,64.9%
431,zilong001,10621,17009,兔子.CHN.,pretender,07.04.2021,62.4%
335,aircraftcarrier,11706,18781,兔子.CHN.,Hermit,07.12.2021,62.3%
1213,liu001001,5858,9536,兔子.CHN.,讨债办,06.29.2021,61.4%
1658,imanoob,4505,7336,兔子.CHN.,pretender,07.08.2021,61.4%


In [63]:
# 荣誉舰队
rongyu <- fleet_power('新荣誉')
rongyu

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1613,guxiaoqiang,4620,7840,新荣誉,虎贲军,07.11.2021,58.9%
2062,hsjj005,3581,6184,新荣誉,天下,05.29.2021,57.9%
1587,zeon0079,4682,8255,新荣誉,海狼团,07.13.2021,56.7%
2294,leviathan179,3101,5477,新荣誉,海狼团,07.12.2021,56.6%
1931,zwl1972,3889,7085,新荣誉,-,07.12.2021,54.9%
2484,a1843452343,2829,5258,新荣誉,虎贲军,06.04.2021,53.8%
2578,qwe3147,2703,5045,新荣誉,海狼团,07.12.2021,53.6%
713,haksyx2003,8316,15525,新荣誉,海狼团,07.12.2021,53.6%
2528,godissatan,2764,5163,新荣誉,虎贲军,07.13.2021,53.5%


In [64]:
# 中华联合舰队
zhonglian <- fleet_power('中华联合舰队')
zhonglian

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1203,liming,5891,9069,中华联合舰队,-,07.13.2021,65.0%
1229,nfship001,5815,9747,中华联合舰队,-,07.12.2021,59.7%
1819,ssdy2736,4129,6944,中华联合舰队,海魂,06.06.2021,59.5%
67,ak47ok,20012,34117,中华联合舰队,中华联合二队,07.13.2021,58.7%
442,daiyinjj,10519,17948,中华联合舰队,中华联合一队,07.12.2021,58.6%
326,hsx330136,11797,20169,中华联合舰队,-,07.11.2021,58.5%
1652,qq95568466,4517,7879,中华联合舰队,-,07.09.2021,57.3%
1316,lee1,5548,9678,中华联合舰队,-,07.04.2021,57.3%
453,down360,10365,18188,中华联合舰队,-,07.12.2021,57.0%


In [65]:
# The Guardians Fleet
tgf <- fleet_power('Dango Daikazoku')
tgf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
243,jedizorro,13161,17264,Dango Daikazoku,-,07.12.2021,76.2%
651,hanyizhou,8624,13137,Dango Daikazoku,-,07.04.2021,65.6%
1793,kf222,4191,6905,Dango Daikazoku,Dragon,07.12.2021,60.7%
390,yixi,11136,18880,Dango Daikazoku,Dragon,07.12.2021,59.0%
309,bibber,12046,20600,Dango Daikazoku,-,07.12.2021,58.5%
1564,yupeng1982,4756,8137,Dango Daikazoku,Dragon,06.27.2021,58.4%
10030,iloveasu,7874,13496,Dango Daikazoku,-,07.13.2021,58.3%
411,ttlz,10912,18761,Dango Daikazoku,Dragon,07.02.2021,58.2%
409,skybear991,10925,18821,Dango Daikazoku,Dragon,07.12.2021,58.0%


In [66]:
# Team Pro Bots
tpb <- fleet_power('TEAM PRO BOTS')
tpb

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1478,pr_ingress,5001,6487,TEAM PRO BOTS,-,07.12.2021,77.1%
947,vosu,6958,9696,TEAM PRO BOTS,TheBestOfBest,07.04.2021,71.8%
353,ded111,11522,16603,TEAM PRO BOTS,$CHMUTZ,07.13.2021,69.4%
548,portunus,9406,14009,TEAM PRO BOTS,$CHMUTZ,07.04.2021,67.1%
32,painlesskiss,24103,36764,TEAM PRO BOTS,-,07.13.2021,65.6%
249,quendil,13082,21165,TEAM PRO BOTS,-,07.13.2021,61.8%
767,schoko,8022,13200,TEAM PRO BOTS,$CHMUTZ,07.13.2021,60.8%
336,hollowman_eu,11686,20152,TEAM PRO BOTS,$CHMUTZ,07.12.2021,58.0%
1080,bestofchill,6398,11112,TEAM PRO BOTS,$CHMUTZ,07.04.2021,57.6%


In [67]:
# BAMF Fleet
bamf <- fleet_power('BAMF FLEET')
bamf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1438,kenvahnz,5118,7381,BAMF FLEET,NERFED,07.13.2021,69.3%
694,ooh_kill_em,8421,12150,BAMF FLEET,Legacy,07.11.2021,69.3%
80,tappo01,19008,27986,BAMF FLEET,DooM,06.28.2021,67.9%
196,koll_,14378,21525,BAMF FLEET,Legacy,07.04.2021,66.8%
10097,lordfail,8977,13625,BAMF FLEET,DooM,06.25.2021,65.9%
1,ms06zaku,60112,92396,BAMF FLEET,BULLS,07.12.2021,65.1%
338,saladrex,11670,18448,BAMF FLEET,NERFED,06.09.2021,63.3%
108,wilsonjordan,16874,26900,BAMF FLEET,#Karma,05.01.2021,62.7%
222,admiralkoth,13731,21950,BAMF FLEET,NERFED,05.15.2021,62.6%


In [68]:
# Russian Alliance
ra <- fleet_power('Russian Alliance')
ra

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
934,nuke666,7065,9005,Russian Alliance,AborzSQ,07.04.2021,78.5%
890,betep,7262,9902,Russian Alliance,Nakhimov,07.13.2021,73.3%
1763,adorius,4256,6717,Russian Alliance,AborzSQ,07.11.2021,63.4%
464,osn3179,10261,16285,Russian Alliance,AborzSQ,07.11.2021,63.0%
814,xxvetalxx,7731,12340,Russian Alliance,Makarov,07.11.2021,62.6%
689,syava_mehan,8436,13676,Russian Alliance,-,07.04.2021,61.7%
1470,varangian,5020,8158,Russian Alliance,-,07.13.2021,61.5%
2163,gother_nfru,3354,5622,Russian Alliance,-,07.06.2021,59.7%
136,reisereise,15979,26909,Russian Alliance,AborzSQ,07.11.2021,59.4%


In [69]:
# 中国蓝舰队
zgl <- fleet_power('中国蓝舰队')
zgl

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
731,wang82840506,8264,13634,中国蓝舰队,☆ ☆ ☆,07.12.2021,60.6%
1779,zjtz000000,4225,6980,中国蓝舰队,☆ ☆ ☆,07.13.2021,60.5%
1045,jovacn,6518,11371,中国蓝舰队,★ ☆ ★,07.12.2021,57.3%
1791,f9977,4193,7394,中国蓝舰队,★ ★ ★,07.12.2021,56.7%
378,wang811019,11249,19847,中国蓝舰队,-,06.10.2021,56.7%
242,kxjh,13166,23694,中国蓝舰队,★ ★ ☆,06.15.2021,55.6%
1929,e9977,3893,7028,中国蓝舰队,☆ ☆ ☆,07.12.2021,55.4%
1398,solo3162,5245,9540,中国蓝舰队,★ ★ ★,07.12.2021,55.0%
134,hz111111,16061,29229,中国蓝舰队,☆ ☆ ☆,07.12.2021,54.9%


In [70]:
# 北方舰队
bf <- fleet_power('北方舰队')
bf

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1513,dongbeiwang,4895,7530,北方舰队,Odin,07.11.2021,65.0%
461,bin2014,10286,16610,北方舰队,THOR,07.12.2021,61.9%
498,bfhjzrb1319,9909,16526,北方舰队,-,07.12.2021,60.0%
2071,langhero,3565,6185,北方舰队,THOR,07.12.2021,57.6%
2028,andy_366,3664,6409,北方舰队,Odin,07.12.2021,57.2%
1507,gbsb8,4911,8624,北方舰队,北方天骄,07.08.2021,56.9%
977,abcdkent,6837,12316,北方舰队,THOR,07.12.2021,55.5%
1212,playab,5858,10613,北方舰队,-,07.10.2021,55.2%
2360,yaoyaopi,3005,5606,北方舰队,THOR,07.12.2021,53.6%


In [71]:
# Ostseepiraten Fleet
osp <- fleet_power('Ostseepiraten')
osp

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
182,steuben_eu,14600,22575,Ostseepiraten,Stoertebeker,05.02.2021,64.7%
2174,ikken3,3332,5476,Ostseepiraten,P_Beneke,07.12.2021,60.8%
629,ikken4,8764,14955,Ostseepiraten,P_Beneke,07.12.2021,58.6%
10088,pamel,10255,17703,Ostseepiraten,-,07.07.2021,57.9%
164,bluestar_eu,15090,26209,Ostseepiraten,corsair,07.12.2021,57.6%
1299,kpt_jack,5586,9993,Ostseepiraten,Stoertebeker,07.01.2021,55.9%
1226,hippo83,5827,10463,Ostseepiraten,P_Beneke,05.23.2021,55.7%
658,jolly_roger2,8593,15620,Ostseepiraten,Rum_u_Ehre,07.12.2021,55.0%
1101,flausmaus,6311,11475,Ostseepiraten,Rum_u_Ehre,07.04.2021,55.0%


In [72]:
# REGALIA
rgla <- fleet_power('REGALIA')
rgla

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1106,joooooo,6282,7864,REGALIA,-,07.12.2021,79.9%
2178,deredador,3328,5834,REGALIA,-,07.11.2021,57.0%
502,unosco,9866,18502,REGALIA,-,07.12.2021,53.3%
521,deseo5,9634,18232,REGALIA,-,07.12.2021,52.8%
1065,translei,6436,12435,REGALIA,-,07.11.2021,51.8%
2580,entroyook,2700,5260,REGALIA,-,07.13.2021,51.3%
1175,yoshyy,5983,11732,REGALIA,-,07.12.2021,51.0%
219,loquic12,13777,27103,REGALIA,Renovation,07.12.2021,50.8%
1323,shyk,5530,10903,REGALIA,-,07.11.2021,50.7%


In [73]:
# ARMADA CHILENA
ach <- fleet_power('ARMADA CHILENA')
ach

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
178,makuvex,14663,27188,ARMADA CHILENA,-,06.13.2021,53.9%
412,lmario666,10901,20789,ARMADA CHILENA,-,07.04.2021,52.4%
1753,carlosii,4273,8261,ARMADA CHILENA,"O""Higgins",07.03.2021,51.7%
1322,xeloborg89,5535,10792,ARMADA CHILENA,-,05.27.2021,51.3%
383,klython1,11196,21890,ARMADA CHILENA,-,07.11.2021,51.1%
1723,saharas,4343,8634,ARMADA CHILENA,-,06.27.2021,50.3%
176,kudoalex,14691,29299,ARMADA CHILENA,"O""Higgins",07.11.2021,50.1%
2374,jecho,2984,5964,ARMADA CHILENA,-,07.11.2021,50.0%
527,kiros,9596,19211,ARMADA CHILENA,-,07.11.2021,50.0%


In [74]:
# Flota Chile
fch <- fleet_power('Flota Chile')
fch

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
187,cuadrad,14557,26128,Flota Chile,-,07.12.2021,55.7%
1198,kulebrakai,5908,10818,Flota Chile,-,07.11.2021,54.6%
372,xaloooo,11276,21167,Flota Chile,-,06.10.2021,53.3%
1096,gabriel_03,6332,11966,Flota Chile,Nanox,06.22.2021,52.9%
1099,chilotecocoa,6319,11944,Flota Chile,-,06.27.2021,52.9%
1577,breaka,4710,8911,Flota Chile,-,07.13.2021,52.9%
593,nitroxx,9053,17183,Flota Chile,ALDEA,06.27.2021,52.7%
758,elite_,8077,15420,Flota Chile,-,07.12.2021,52.4%
305,azsnar,12137,23175,Flota Chile,-,07.12.2021,52.4%


In [75]:
# Imperium
imp <- fleet_power('Imperium')
imp

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
71,aildinchen,19550,33081,Imperium,Aggressor,07.10.2021,59.1%
587,delta56,9130,15764,Imperium,Veterans,07.13.2021,57.9%
656,ssepan,8601,14976,Imperium,-,07.13.2021,57.4%
49,slayer_z,22225,39255,Imperium,MFDM,07.02.2021,56.6%
1273,nbarloya,5668,10113,Imperium,Aggressor,07.12.2021,56.0%
774,sema433,7961,14406,Imperium,Veterans,07.13.2021,55.3%
102,tmac2,17297,31425,Imperium,-,05.16.2021,55.0%
542,hussy,9451,17388,Imperium,MFDM,07.12.2021,54.4%
745,benbonebone,8158,15182,Imperium,-,07.12.2021,53.7%


In [76]:
# Armored Angels
aa <- fleet_power('Armored Angels')
aa

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
358,rainbow112,11462,18300,Armored Angels,Titans SQ,07.10.2021,62.6%
529,nager,9588,15876,Armored Angels,-,05.08.2021,60.4%
240,wishbonej,13230,23225,Armored Angels,-,05.14.2021,57.0%
173,will211,14775,27830,Armored Angels,-,07.12.2021,53.1%
200,hideship12,14235,26991,Armored Angels,Titans SQ,06.26.2021,52.7%
607,zemex,8947,17080,Armored Angels,Titans SQ,07.12.2021,52.4%
510,admstef,9759,18797,Armored Angels,-,07.07.2021,51.9%
1055,wawa300,6467,12632,Armored Angels,Titans SQ,07.10.2021,51.2%
123,djr63,16381,32061,Armored Angels,Saintz SQ,07.04.2021,51.1%


In [77]:
# HELL BOYS
hb <- fleet_power('HELL BOYS')
hb

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1153,_vicon_,6080,9802,HELL BOYS,ZETAS,06.29.2021,62.0%
21,altsein,26005,44322,HELL BOYS,-,07.13.2021,58.7%
147,hitachi88,15615,26802,HELL BOYS,ZETAS,07.12.2021,58.3%
29,patatoo,24588,42484,HELL BOYS,-,06.13.2021,57.9%
199,det_ka,14256,24908,HELL BOYS,ZETAS,07.12.2021,57.2%
337,dakota775,11675,20512,HELL BOYS,ZETAS,06.06.2021,56.9%
520,andr2776,9665,17199,HELL BOYS,ZETAS,07.12.2021,56.2%
429,xelementx,10659,19223,HELL BOYS,-,07.12.2021,55.4%
771,fenix69,7996,14744,HELL BOYS,LIONS,06.27.2021,54.2%


In [78]:
# Reiwa Fleet
fleet_power('Reiwa Fleet')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2183,y4427,3320,5051,Reiwa Fleet,-,07.12.2021,65.7%
2133,wanco2121,3403,5523,Reiwa Fleet,-,05.22.2021,61.6%
2244,aituki,3192,5185,Reiwa Fleet,-,07.03.2021,61.6%
105,zatti,17182,27968,Reiwa Fleet,-,06.08.2021,61.4%
1657,hesky,4508,8005,Reiwa Fleet,-,07.12.2021,56.3%
1059,sigemori,6448,12035,Reiwa Fleet,-,06.30.2021,53.6%
958,captainmaru,6922,13111,Reiwa Fleet,-,07.12.2021,52.8%
2620,cx817us,2640,5043,Reiwa Fleet,-,07.13.2021,52.3%
2156,wakasa3n,3368,6569,Reiwa Fleet,-,07.03.2021,51.3%


In [89]:
# Jedi Fleet
fleet_power('Jedi Fleet')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
12,jackaljl,29244,49458,Jedi Fleet,-,07.12.2021,59.1%
1891,sickman,4003,7012,Jedi Fleet,-,05.22.2021,57.1%
1802,black101,4169,7824,Jedi Fleet,-,06.13.2021,53.3%
370,oopsitorpedu,11337,21664,Jedi Fleet,-,05.04.2021,52.3%
1086,rutgersrocks,6372,12369,Jedi Fleet,-,05.21.2021,51.5%
1904,hotshootgg,3975,8090,Jedi Fleet,-,06.19.2021,49.1%
1570,langan,4731,10003,Jedi Fleet,-,06.22.2021,47.3%
1988,tk0rahl,3775,7996,Jedi Fleet,Sith Empire,07.08.2021,47.2%
2961,tacnuke,2222,5252,Jedi Fleet,-,06.23.2021,42.3%


In [90]:
# TeamKorea
fleet_power('TeamKorea')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2361,mndusers,3004,5051,TeamKorea,Korea,07.12.2021,59.5%
418,timmerman,10833,18759,TeamKorea,-,07.12.2021,57.7%
1659,seedream,4503,8517,TeamKorea,-,07.11.2021,52.9%
1895,bnbchd,3993,7597,TeamKorea,-,07.12.2021,52.6%
1047,torabin,6505,12379,TeamKorea,-,07.12.2021,52.5%
1715,doria1611,4371,8550,TeamKorea,-,06.27.2021,51.1%
2074,windcirclet,3561,6992,TeamKorea,Korea,07.12.2021,50.9%
1848,jkl3188,4087,8174,TeamKorea,Korea,06.20.2021,50.0%
2598,piraten1,2678,5422,TeamKorea,-,07.12.2021,49.4%


In [91]:
# HBZ联合舰队
fleet_power('HBZ联合舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
236,cj1230,13398,23979,HBZ联合舰队,-,07.12.2021,55.9%
2330,vincent379,3051,5764,HBZ联合舰队,-,07.09.2021,52.9%


In [92]:
# 乾坤舰队
fleet_power('乾坤舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
490,tjdy,9990,17490,乾坤舰队,-,07.12.2021,57.1%
2092,hundun01,3500,6314,乾坤舰队,-,07.12.2021,55.4%
2301,ninohui218,3091,5807,乾坤舰队,-,07.13.2021,53.2%
863,zhkim72,7348,14029,乾坤舰队,-,07.12.2021,52.4%
1127,qk015,6214,11998,乾坤舰队,-,07.02.2021,51.8%
1726,sujian521,4335,8487,乾坤舰队,-,07.12.2021,51.1%
1990,miao1974310,3765,7835,乾坤舰队,-,06.17.2021,48.1%
1340,yahhor,5463,11403,乾坤舰队,-,07.12.2021,47.9%


In [93]:
# 皇家零零七
fleet_power('皇家零零七')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
765,kpwlt0013,8040,14519,皇家零零七,情谊,07.11.2021,55.4%
2414,bzsd,2926,5304,皇家零零七,情谊,07.12.2021,55.2%
1816,huizi1616,4131,7710,皇家零零七,情谊,07.12.2021,53.6%
722,xiaonan2018,8280,15654,皇家零零七,情谊,07.12.2021,52.9%
930,sata100,7078,13386,皇家零零七,情谊,07.13.2021,52.9%
2603,ziri2121,2666,5138,皇家零零七,-,07.12.2021,51.9%
1639,huilangooo,4557,8819,皇家零零七,情谊,06.24.2021,51.7%
1309,jian2524,5568,11071,皇家零零七,情谊,07.12.2021,50.3%
2729,huilang,2498,5199,皇家零零七,情谊,05.21.2021,48.0%


In [102]:
# 拆迁办
fleet_power('拆迁办')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
965,hyz980517,6902,11229,拆迁办,Logistics,06.16.2021,61.5%
2098,liu001002,3486,5794,拆迁办,-,06.29.2021,60.2%
2585,nulei01,2698,5146,拆迁办,Logistics,07.12.2021,52.4%
648,linhai12,8647,16569,拆迁办,荣耀,07.13.2021,52.2%
2499,gt3000,2808,5384,拆迁办,朱雀,07.12.2021,52.2%
822,hw0871,7674,14742,拆迁办,朱雀,07.11.2021,52.1%
942,pingtouge,6971,13414,拆迁办,玄武,07.10.2021,52.0%
2507,ridgycs,2788,5390,拆迁办,玄武,07.12.2021,51.7%
808,baozhu001,7769,15052,拆迁办,朱雀,07.12.2021,51.6%


In [100]:
# Homeless
fleet_power('-')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1736,baozi77,4310,5585,-,-,07.12.2021,77.2%
247,k435345,13128,18023,-,-,07.04.2021,72.8%
244,greattak,13158,18252,-,-,07.12.2021,72.1%
1231,znfecu,5811,8151,-,勇敢天下,07.12.2021,71.3%
1158,magey2,6049,8957,-,-,05.10.2021,67.5%
69,huitainan_a,19624,29417,-,-,07.12.2021,66.7%
589,laopu,9108,13682,-,-,07.13.2021,66.6%
912,fs4025paz,7165,10777,-,-,07.12.2021,66.5%
332,berserker120,11734,18048,-,-,07.12.2021,65.0%


In [85]:
as.list(unique(nf_data['fleet']))

$fleet
 [1] "BAMF FLEET"       "Russian Alliance" "兔子.CHN."        "-"               
 [5] "Ostseepiraten"    "HELL BOYS"        "TEAM PRO BOTS"    "Imperium"        
 [9] "Jedi Fleet"       "FreeThaiMoveMent" "Armada Argentina" "Valhalla"        
[13] "Dt. Flottenbund"  "新荣誉"           "Flota Peruana"    "Armored Angels"  
[17] "大风云舰队"       "Reiwa Fleet"      "Flota Chile"      "中华联合舰队"    
[21] "C.O.H."           "ULTRAS+"          "中国蓝舰队"       "ARMADA CHILENA"  
[25] "海风舰队"         "VNFRIENDS"        "REGALIA"          "復仇者大聯盟"    
[29] "深海舰队"         "E.D.F"            "HBZ联合舰队"      "Dango Daikazoku" 
[33] "War Navy"         "TeamKorea"        "北方舰队"         "TURKIYE"         
[37] "Great_Sharks"     "Poseidon"         "乾坤舰队"         "尖刀连"          
[41] "第四力量"         "Flota Gallega"    "名將堂舰队"       "-Killerigel-"    
[45] "LEYENDA HISPANA"  "007舰队"          "拆迁办"           "TeamNF"          
[49] "皇家零零七"       "琥 珀"            "首战"             "=中国蓝="        
[53] "ARGONAUTS"        "The First Shock"  "REGIA.MARINA.ITA" "齐云舰队"        
[57] "云融"             "第七联合舰队"     "猎杀2020"         "荣誉"            
[61] "武林堂"           "Dark Side"        "水手集团总公司"   "烟民"            
[65] "I.J.N"            "UN联合舰队"       "娱乐派对"         "戦斧"            
[69] "旗艦聯合"         "Ｖ＆Ｃ海盗团"     "海狼"             "CRUZ DEL SUR"    
[73] "第九宫舰队"       "北京舰队"         "大秦帝国舰队"     "八一海军"        
[77] "铁十字"           "武汉联合舰队"     "德意志帝国海军"   "SS帝国公海舰队"  
[81] "怒海远征舰队"     "Arg-1982"         "兔子.CHN"         "H_o_D"           
[85] "彩虹舰队"         "新荣誉舰队"       "正和舰队"         "M.F Fleet"       
[89] "南海舰队"         "鹰击舰队"         "SeaWolves"        "炎黄海军"        
[93] "north sea"        "达瓦里氏冲锋队"   "自由国度"         "第四力量舰队"    
[97] "World Team"       "戮虎-未来之光"    "三国无双"

In [86]:
as.list(unique(nf_data['squad']))

$squad
  [1] "BULLS"            "Nakhimov"         "-"               
  [4] "Crusaders"        "Sith"             "GLORIUS"         
  [7] "MFDM"             "HisThaiMajestyS"  "IMMORTALS"       
 [10] "P_Beneke"         "DooM"             "Veterans"        
 [13] "corsair"          "Legacy"           "BAP VIP"         
 [16] "USHAK0V"          "Titans SQ"        "ZETAS"           
 [19] "ALDEA"            "NERFED"           "中华联合二队"    
 [22] "ARA SQ Charlie"   "Aggressor"        "Sq Charlie"      
 [25] "Dragons"          "Sagrados"         "#Karma"          
 [28] "LIONS"            "Saintz SQ"        "$CHMUTZ"         
 [31] "Stoertebeker"     "☆ ☆ ☆"            "AborzSQ"         
 [34] "World"            "BAP_SUR"          "=OLAYA="         
 [37] "THE IMMORTALS"    "讨债办"           "Loewengeschwader"
 [40] "O\"Higgins"       "Tigergeschwader"  "HUSARES"         
 [43] "Rock Squad"       ".-Lord-"          "pretender"       
 [46] "Renovation"       "☆ ★ ☆"            "★ ★ ☆"           
 [49] "BoB_Soeldner"     "_ECHO"            "_ALPHA"          
 [52] "jancio dywizjon"  "Team Pollera"     "Hermit"          
 [55] "海風奔馳"         "Phil Tin Cans"    "Rum_u_Ehre"      
 [58] "SQ. CHILE"        "Dragon"           "Sons of anarchy" 
 [61] "勇敢天下"         "TATRY"            "bangkok"         
 [64] "SUPREMO"          "中华联合一队"     "Thundercats"     
 [67] "THOR"             "Old Boy"          "ARA SQ Delta"    
 [70] "海風聚義"         "Pure Darkness"    "F_Drake"         
 [73] "巅锋舰队"         "Bravery"          "雪豹旅"          
 [76] "Andes"            "Makarov"          "深蓝战队"        
 [79] "★ ★ ★"            "荣耀"             "海狼团"          
 [82] "情谊"             "虎贲军"           "远征军"          
 [85] "-=HAWK=-"         "Nanox"            "朱雀"            
 [88] "暴击小分队"       "-=☆☆☆=-"          "ARA SQ Bravo"    
 [91] ""                 "玄武"             "TheBestOfBest"   
 [94] "龍城"             "AYYY LMAO"        "Logistics"       
 [97] "海浪"             "深渊"             "R.O.C"           
[100] "海风之096老炮"    "共和国之鹰1"      "★ ☆ ★"           
[103] "勛章"             "BLACKFLEET"       "Sq C.O.H."       
[106] "Fr.Kiss"          "Hydra"            "〓★〓"           
[109] "M.F共和国之鹰"    "=L&G="            "=B.DemonsSharks" 
[112] "REGIA"            "Odin"             "Legion"          
[115] "-=★=-"            "炮灰团"           "海風"            
[118] "BUCETAO"          "ＺＧＬ"           "兄弟盟"          
[121] "特混编队"         "Bikers"           "北纬37度"        
[124] "扬帆起航"         "特洛伊"           "English Squad"   
[127] "=名将堂="         "北方天骄"         "Cheewooda"       
[130] "ARA SQ Mercs"     "Nuke From Orbit"  "Kyojano"         
[133] "〓☆〓"            "海魂"             "Korea"           
[136] "荣誉团"           "=FG="             "Freedom Fighter" 
[139] "Sith Empire"      "LOKI"             "无畏"            
[142] "天下"             "雷霆"             "Ｖ＆Ｃ"          
[145] "Sea Wolves"       "Grupo Especial"   "Inter union"     
[148] "大秦帝国虎贲队"   "光荣"             "TeamNF Monitor"  
[151] "战舰航母"         "鸭王"             "名将堂雷神"      
[154] "拂晓"             "名将堂战狼"       "JUICY FRUIT"     
[157] "德意志黑人抬棺"   "德意志王牌战队"   "吾之荣誉即忠诚"  
[160] "幻象騎士團"       "深海"             "YAMATOdama"      
[163] "自由战士"         "帝王"             "中华太平洋舰队"  
[166] "Fist"             "扬威四海"         "大风云"          
[169] "USSLaJIWaJ"       "Niko nii~"        "Juliamo"         
[172] "Ushakov"          "ARDITI"           "孤军"            
[175] "天驕"             "F.C.J"            "专业_批发核弹头" 
[178] "大秦雄鹰战队"     "AsturSpain"       "汁男"            
[181] "南海舰队"         "Argentina"        "八一海军"        
[184] "Armored Navy"     "颶風"             "丽春院"          
[187] "统帅堂"           "海上雄狮"         "Team Germany"    
[190] "〓潇★湘〓"        "奉天"             "广东舰队古惑仔"  
[193] "银河舰队"         "锦帆"             "SS帝国公海舰队"  
[196] "龙旗飘飘"         "南宁战队"         "恶魔之魂"        
[199] "彩虹舰队"         "EuropeanCup"      "Jolly_Roger"     
[202] "秃鷹軍團"         "Shark Fin Soup"   "1st Fleet"       
[

In [87]:
# 芜湖连坑
squad_power('兄弟盟')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1367,rus1976,5348,11026,-,兄弟盟,07.12.2021,48.5%
1471,xc87995348,5019,10932,-,兄弟盟,07.13.2021,45.9%


In [94]:
# 巅锋舰队
squad_power('巅锋舰队')

,id,wins,battles,fleet,squad,activity,win_rate
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
2255,shuyang66,3163,5946,-,巅锋舰队,07.12.2021,53.2%
532,shuyang88,9568,18596,-,巅锋舰队,07.12.2021,51.5%
10098,laixilaixi888,4289,8491,-,巅锋舰队,07.11.2021,50.5%
2333,lufanss,3050,6054,-,巅锋舰队,07.11.2021,50.4%
1145,tmdsb,6101,12143,-,巅锋舰队,07.11.2021,50.2%
